## Training a character language model and studying various ways of generating text

**Author: Souhail OUMAMA (souhail.oumama@telecom-paris.fr)**  
**Based on Original lab by : Matthieu LABEAU (matthieu.labeau@telecom-paris.fr)**

## Objectives:

- We will train a network to predict a next character given an input sequence of characters, and use it to generate new sequences.
- We will strictly work with local (and not structured) prediction - however, we will look into a relatively simple heuristic to improve the "structure": *beam search*. We can also try to improve generation with other methods: *temperature* sampling, *top-k* sampling, *top-p* sampling,
- We will use ```keras```to build the model based on a LSTM, which will use simple features (one-hot vector representing previous characters) to predict the next characters. We will use a small model to avoid training for too long.
- We will use a small dataset (poetry, from project Gutenberg) - you can use any data you prefer, as long as you are able to train the model on it. 
- Even with a small dataset and a small model, training may be long. If you can use a computing infrastructure, like Google colab, it may be more practical - and you probably can obtain better results by using a bigger model and a larger dataset.

#### Obtaining the data 
- We download directly the ebook from project Gutenberg - you can get any other text you would prefer. 

In [1]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt

URL transformed to HTTPS due to an HSTS policy
--2021-11-30 22:29:03--  https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573338 (4.4M) [text/plain]
Saving to: ‘shakespeare_input.txt.3’

shakespeare_input.t 100%[===================>]   4.36M  3.70MB/s    in 1.2s    

2021-11-30 22:29:04 (3.70 MB/s) - ‘shakespeare_input.txt.3’ saved [4573338/4573338]



In [8]:
from keras.utils.data_utils import get_file
url = 'http://www.gutenberg.org/cache/epub/6099/pg6099.txt'
path = get_file('pg6099.txt', origin=url)

f = open(path, 'r' , encoding = 'utf8')
lines = f.readlines()
text = []

start = False
for line in lines:
    if("START OF THIS PROJECT GUTENBERG EBOOK LES FLEURS DU MAL" in line and start==False):
        start = True
    if("END OF THIS PROJECT GUTENBERG EBOOK LES FLEURS DU MAL" in line):
        break
    if(start==False or len(line) == 0):
        continue
    text.append(line)

f.close()
text = " ".join(text)
voc_chars = sorted(set([c for c in text]))
nb_chars = len(voc_chars)

In [92]:
print(text[20000:21000])

 ces plaintes,
   Ces extases, ces cris, ces pleurs, ces _Te Deum,_
   Sont un écho redit par mille labyrinthes;
   C'est pour les coeurs mortels un divin opium.
 
   C'est un cri répété par mille sentinelles,
   Un ordre renvoyé par mille porte-voix;
   C'est un phare allumé sur mille citadelles,
   Un appel de chasseurs perdus dans les grands bois!
 
   Car c'est vraiment, Seigneur, le meilleur témoignage
   Que nous puissions donner de notre dignité
   Que cet ardent sanglot qui roule d'âge en âge
   Et vient mourir au bord de votre éternité!
 
 
 
 
   LA MUSE VENALE
 
 
   O Muse de mon coeur, amante des palais,
   Auras-tu, quand Janvier lâchera ses Borées,
   Durant les noirs ennuis des neigeuses soirées,
   Un tison pour chauffer tes deux pieds violets?
 
   Ranimeras-tu donc tes épaules marbrées
   Aux nocturnes rayons qui percent les volets?
   Sentant ta bourse à sec autant que ton palais,
   Récolteras-tu l'or des voûtes azurées?
 
   Il te faut, pour gagner ton pain de cha

#### Keeping track of possible characters
- Using a ```set```, create a sorted list of possible characters
- Create two dictionnaries, having characters and corresponding indexes as {key: value}, and reverse.

Example:

```chars = [a, b, c]```

```chars_indices = {a: 0, b: 1, c: 2}```

```indices_chars = {0: a, 1: b, 2: c}```

In [67]:
print('Corpus length:', len(text))

chars = sorted(list(set(text)))
print('Total number of characters:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Corpus length: 160576
Total number of characters: 92


#### Creating training data
- We will represent characters using *one-hot vectors*. Hence, the i-th character of n possible characters will be represented by a vector of length $n$, containing $0$ expect for a $1$ in position $i$. Following our previous examples, ```a = [1, 0, 0]``` and ```b = [0, 1, 0]```. 
- Hence, a sequence of characters is a list of one-hot vectors. Our goal will be to predict, given an input sequence of fixed length (here, this length is given by ```maximum_seq_length```) the next character. Hence, we need to build two lists: ```sentences```, containing the input sequences, and ```next_char``` the characters to be predicted. 
- We do not necessarily need to take all possible sequences. We can select one every ```time_step``` steps.

Example: Using the previous dictionnaries, the sequence:
```'acabbaccaabba'``` with ```maximum_seq_length = 4``` and ```time_step = 2``` would give the following lists: 

```sentences = ['acab', 'abba', 'bacc', 'ccaa', 'aabb']```

```next_char = ['b', 'c', 'a', 'b', 'a']```

In [71]:
maximum_seq_length = 24
time_step = 1
sentences = []
next_char = []
for i in range(0, len(text) - maximum_seq_length, time_step):
    sentences.append(text[i: i + maximum_seq_length])
    next_char.append(text[i + maximum_seq_length])
print('Number of Sequences:', len(sentences))

Number of Sequences: 160552


In [11]:
import numpy as np
import random
import sys

#### Creating training tensors
- We need to transform these lists into tensors, using one-hot vectors to represent characters.
- We will need 3 dimensions for the training examples from ```sentences```: the number of examples, the length of the sequence, and the dimension of the one-hot vector
- This is reduced to 2 dimensions for the ```next_char```: number of examples and one-hot vector. 

Example: the previous ```sentences``` would become:

```X = [[[1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]],
      [[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]],
      [[0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1]],
      [[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]],
      [[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0]]] ```
       
```y = [[0, 1, 0],
      [0, 0, 1],
      [1, 0, 0],
      [0, 1, 0],
      [1, 0, 0]]```

In [12]:
X = np.zeros((len(sentences), maximum_seq_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_char[i]]] = 1

#### Implement the model
- We need to create a LSTM model that takes directly out inputs from ```X``` and try to predict one-hot vectors from ```y```.
- The model should be made with a LSTM layer, and a Dense layer followed by a softmax activation function. 
- The most important arguments are the ```input_shape``` for the LSTM layer and the proper size for the Dense layer. How do we find them out ? 
- We can use a moderate size for the hidden states for the LSTM. 
- We use the ```categorical_crossentropy``` loss, with the optimizer of our preference (for example, ```adam```).

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.callbacks import LambdaCallback, EarlyStopping

In [19]:
model = Sequential()
model.add(LSTM(256, input_shape=(maximum_seq_length, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

#### Create a function to generate text with our model
- We use the output of our model (a vector of probabilities on characters) to select the next most probable character (with the ```argmax``` function)
- We need to transform an input text into an input tensor, as before (taking the right length, the last ```maximum_seq_length``` characters)
- We need to transform back the most probable index into a character and add it to our text.
- This must be looped ```num_generated``` times, each time obtaining a new input tensor from the new input sequence (which has the character we previously predicted at the end !)

The following function (```end_epoch_generate```) is here to facilitate automatic generation at the end of each epoch, so you can monitor of generation changes as the model trains. It calls the ```generate_next``` function upon each sequence of text in ```texts_ex```. The only element in this list right now comes from the training data - you can add your own.

In [15]:
def generate_next(model, text, num_generated=120):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.
        predictions = model.predict(x, verbose=0)[0]
        next_index = np.argmax(predictions)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)
    
def end_epoch_generate(epoch, _):
    print('\n Generating text after epoch: %d' % (epoch+1))
    texts_ex = ["La sottise, l'erreur, le péché"]
    for text in texts_ex:
        sample = generate_next(model, text)
        print('%s' % (sample))

In [16]:
text_ex = "La sottise, l'erreur, le péché"
generate_next(model, text_ex)

"La sottise, l'erreur, le péchéBBtYYYUF__F__7bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"

In [20]:
model.fit(X, y,
          batch_size=128,
          epochs=10,
          validation_split = 0.2,
          callbacks=[LambdaCallback(on_epoch_end=end_epoch_generate)])

Epoch 1/10
1002/1004 [============================>.] - ETA: 0s - loss: 2.5601
 Generating text after epoch: 1
La sottise, l'erreur, le péchés de pais de pais de pais de pais de pais de pais de pais de pais de pais de pais de pais de pais de pais de pais de pai
1004/1004 [==============================] - 24s 21ms/step - loss: 2.5594 - val_loss: 2.1404
Epoch 2/10
1001/1004 [============================>.] - ETA: 0s - loss: 2.0649
 Generating text after epoch: 2
La sottise, l'erreur, le péchés de l'ant le son son son son son son son son son son son son son son son son son son son son son son son son son son so
1004/1004 [==============================] - 21s 21ms/step - loss: 2.0648 - val_loss: 1.9828
Epoch 3/10
1003/1004 [============================>.] - ETA: 0s - loss: 1.9400
 Generating text after epoch: 3
La sottise, l'erreur, le péchérau,
      autes de le fours de  leurs de les den  de les des de le foure de l'un de le des de les des des de le foure d
1004/1004 [=================

#### Using character embeddings 
- Instead of using one-hot vectors to represent characters, we will now use character embeddings, which are vectors belonging to the same space. 
- We will need as many vectors as there is characters. The input of the network will be simpler, since we will just need to indicate to the model which character is in input. 
- The output does not change: indeed, Keras uses one-hot vectors for the target of the categorical cross-entropy loss. 
Example: the previous example ```sentences``` would now become:
- We need to add a ```Embedding``` layer to the model, with the right input size, and to choose which dimension use for our embeddings. 

```X = [[0, 2, 0, 1],
        [0, 1, 1, 0],
        [1, 0, 2, 2],
        [2, 2, 0, 0],
        [0, 0, 1, 1]]```
       
```y = [[0, 1, 0],
      [0, 0, 1],
      [1, 0, 0],
      [0, 1, 0],
      [1, 0, 0]]```

In [25]:
X_emb = np.zeros((len(sentences), maximum_seq_length), dtype=np.int)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X_emb[i, t] = char_indices[char]

In [26]:
model_emb = Sequential()
model_emb.add(Embedding(len(chars), 32, input_length = maximum_seq_length))
model_emb.add(LSTM(256))
model_emb.add(Dense(len(chars)))
model_emb.add(Activation('softmax'))

model_emb.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
def generate_next(model, text, num_generated=120):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length))
        for t, char in enumerate(sentence):
            x[0, t] = char_indices[char]
        predictions = model.predict(x, verbose=0)[0]
        next_index = np.argmax(predictions)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)
    
def end_epoch_generate(epoch, _):
    print('\n Generating text after epoch: %d' % (epoch+1))
    texts_ex = ["La sottise, l'erreur, le péché"]
    for text in texts_ex:
        sample = generate_next(model_emb, text)
        print('%s' % (sample))

In [31]:
model_emb.fit(X_emb, y,
          batch_size=128,
          epochs=10,
          validation_split = 0.2,
          callbacks=[LambdaCallback(on_epoch_end=end_epoch_generate)])

Epoch 1/10
502/502 [==============================] - ETA: 0s - loss: 1.8807
 Generating text after epoch: 1
La sottise, l'erreur, le péchés de soir de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de l'ain de
502/502 [==============================] - 16s 31ms/step - loss: 1.8807 - val_loss: 1.8452
Epoch 2/10
502/502 [==============================] - ETA: 0s - loss: 1.8049
 Generating text after epoch: 2
La sottise, l'erreur, le péché de le pris de les pleurs des chasses des chasses des chasses des chasses des chasses des chasses des chasses des chasse
502/502 [==============================] - 16s 31ms/step - loss: 1.8049 - val_loss: 1.7899
Epoch 3/10
501/502 [============================>.] - ETA: 0s - loss: 1.7432
 Generating text after epoch: 3
La sottise, l'erreur, le péchés de la morte de la morte de la morte de la morte de la morte de la morte de la morte de la morte de la morte de la mort
502/502 [=============================

#### Sampling with our model
- Now, instead of simply selecting the most probable next character, we would like to be able to draw a sample from the distribution output by the model.
- To better control the generation, we would like to use the argument ```temperature```, to smooth the distribution.
- We will use the ```multinomial``` function from the ```random``` package to draw samples.
- We integrate this into a function ```generate_sample``` that is almost exactly like ```generate_next```.

In [79]:
def reweight(predictions, temperature):
    predictions = np.asarray(predictions).astype('float64')
    log_predictions = np.log(predictions) / temperature
    predictions = np.exp(log_predictions)
    predictions = predictions / np.sum(predictions)
    return predictions   

def sample(predictions, temperature):
    predictions = reweight(predictions, temperature)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

def generate_sample(model, text, num_generated=120, temperature=1.0):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length))
        for t, char in enumerate(sentence):
            x[0, t] = char_indices[char]
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [83]:
print(generate_sample(model_emb, text_ex, temperature = 0.7))

La sottise, l'erreur, le péché la nuit mon jaye et ses bauteurs serres aux; brassiquon,
   Tu connus qui fit le singulier les cepuneux pareurs,
   Des


In [84]:
def sample_top_k(predictions, temperature, k):
    predictions = np.asarray(predictions).astype('float64')
    log_predictions = np.log(predictions) / temperature
    indices_to_remove = log_predictions.argsort()[:k]
    log_predictions[indices_to_remove] = -float('Inf')
    predictions = np.exp(log_predictions)
    predictions = predictions / np.sum(predictions)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)


def generate_sample_top_k(model, text, num_generated=120, temperature=1.0, k=10):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length))
        for t, char in enumerate(sentence):
            x[0, t] = char_indices[char]
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample_top_k(predictions, temperature, k)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [85]:
print(generate_sample_top_k(model_emb, text_ex, temperature = 0.7, k=5))

La sottise, l'erreur, le péché sur le flasse et ses pleurs.
 
   C'est l'on ma moile, harmire et sans soupire
   Sous le lourd tien de la noir et les 


In [86]:
def sample_top_p(predictions, temperature, p):
    predictions = np.asarray(predictions).astype('float64')
    log_predictions = np.log(predictions) / temperature
    predictions = np.exp(log_predictions)
    predictions = predictions / np.sum(predictions)
    
    cum_prob = 0.0
    incr = 0
    indices = predictions.argsort()
    probs = predictions[indices]
    while cum_prob < p:
        cum_prob += probs[incr]
        incr += 1
    indices_to_remove = indices[incr:]
    
    log_predictions[indices_to_remove] = -float('Inf')
    predictions = np.exp(log_predictions)
    predictions = predictions / np.sum(predictions)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)


def generate_sample_top_p(model, text, num_generated=60, temperature=1.0, p=0.9):
    generated = text
    sentence = text[-maximum_seq_length:]
    for i in range(num_generated):
        x = np.zeros((1, maximum_seq_length))
        for t, char in enumerate(sentence):
            x[0, t] = char_indices[char]
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample_top_p(predictions, temperature, p)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return(generated)

In [87]:
print(generate_sample_top_p(model_emb, text_ex.lower(), temperature = 0.7, p = 0.6))

la sottise, l'erreur, le péchérait autome au sentique éternelle,
   Des rends les souirs s


#### Generate text with the beam algorithm
- We need to loop for each character we want to generate, keeping track of the best ```beam_size``` sequences at the most. 
- Besides keeping track of past generated character for each of these ```beam_size``` sequences, we need to keep track of their log-probability.
- This is done by, at each loop, keeping the ```beam_size```best predictions for each of the ```beam_size``` sequences, computing the log-probabilities of the newly formed (```beam_size```)$^2$ , and keeping the overall ```beam_size``` best new sequences.

In [90]:
def generate_beam(model, text, beam_size=16, num_generated=128):
    generated = text
    sentence = text[-maximum_seq_length:]
    # Initialization of the beam with log-probabilities for the sequence
    current_beam = [(0, [], sentence)]

    for l in range(num_generated):
        all_beams = []
        for prob, current_preds, current_input in current_beam:
            x = np.zeros((1, maximum_seq_length))
            for t, char in enumerate(current_input):
                x[0, t] = char_indices[char]
            prediction = model.predict(x=x)[0]
            possible_next_chars = prediction.argsort()[-beam_size:][::-1]
            all_beams += [
                (prob + np.log(prediction[next_index]),
                 current_preds + [next_index],
                 current_input[1:] + indices_char[next_index]   
                )
                for next_index in possible_next_chars]

        current_beam = sorted(all_beams)[-beam_size:]

    return text + ''.join([indices_char[idx] for idx in current_preds])

In [91]:
print(generate_beam(model_emb, text_ex))

La sottise, l'erreur, le péché,
   Comme un coeur d'un coeur et le soir,
   Comme un coeur d'un coeur et le soir,
   Comme un coeur d'un coeur et le soir,
  
